# AES 256 Encrypt & Decrypt


## AES의 특징
 - 입력 평문의 길이는 128 바이트로 고정
 - AES의 기본 연산은 Byte 단위로 수행
 - 사용하는 암호화 키의 길이는 128, 192, 256 중 선택 가능
 - Rigindel 알고리즘은 키의 길이와 입력 평문의 길이를 128, 192, 256 비트 중 선택 가능


In [2]:
pip install pycrypto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 5.0 MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=52a8d9a6cd481c850f4e065aac737875ab32ae280fde561f836a7bcd3c8e9551
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pycrypto


# Padding For BlockSize

암호화를 진행하고자 하는 String 데이터의 기이가 Block Size의 배수가 아닐 때 Padding이 필요하다. AES는 128 bit 즉, 16 Byte로 고정되는데, 다음의 코드로 Padding 처리한다. 

한글의 경우 len(s.encode('utf-8')) 처리가 필요하다.

영문과 기호는 문자당 1 Byte이지만 한글은 2 Byte이기때문이다.

In [7]:
BlockSize = 16 # Init BlockSize (16 Byte)
pad = lambda s:s + (BlockSize - len(s.encode('utf-8')) % BlockSize) * chr(BlockSize - len(s.encode('utf-8')) % BlockSize)
unpad = lambda s:s[:-ord(s[len(s)-1:])]

# AESCipher Class

In [167]:
# AES 256 encryption/decryption using pycrypto library
 
import base64
import hashlib
from Crypto.Cipher import AES
from Crypto import Random
import time

BLOCK_SIZE = 16
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]
 
key = str(int(time.time()))
 
 
class AESCipher:
  def __init__(self, key):
    self.key = key

  def encrypt(raw, key):
    private_key = hashlib.sha256(key.encode("utf-8")).digest()
    raw = pad(raw)
    iv = Random.new().read(AES.block_size)
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return base64.b64encode(iv + cipher.encrypt(raw))
 
 
  def decrypt(enc, key):
    private_key = hashlib.sha256(key.encode("utf-8")).digest()
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(enc[16:]))

In [175]:
plaintext = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."

In [177]:
# First let us encrypt secret message
encrypted = AESCipher.encrypt(plaintext, key)
print(encrypted)
 
# Let us decrypt using our original password
decrypted = AESCipher.decrypt(encrypted, key)
print(bytes.decode(decrypted))

b'HzbvfRA3aF0EQ4XQaqCxNAq3XXDqjS63LQ1Fwr0KeDy2JGzgx4KT//unrI+2nD5BAQbC7vZQExs2R4hYnnIutilsnQa7Sl+fJzi1X16WSmbuHsNp05WTQv5rH+6MU9lp/Q+uYLsw3AKa0CpTyWrRgUxXRptD9C8VF/9ZUWYO1N27NPp7BDAb48ODTglkpEjZkMvQ8OBEKCpRZ0XDNvgJFrdIAN0Z2ABcqFpWugcg+KcOCY574roUSDxGddsMUnAcwieDBMTC/KNXLwlLks1DpRLFx+bd5GzuV3GdUHFg77Qtq1atix5OzDt8+mt22SWYR1+3MHyhAi8wGEYfF4jSNiMIyLgoAikqyGERaaQ/tU05MigJ0eSE5IWCJW1NBPZB6lQeaLrWPd2EspzwbH+k9iItjBxeO55AaibVIw+otmemgPz4HEVuNA2ORyZUB2cPQqJei4EVnUj+tidgsqaJ1eAdWhEM0v7Jqgxd2l01a7szx/+XIwTOugxOH9P9Tq3N48uUgjZNDKpRjil5sGYh+RanPCPuGxhFailVPld/AjxJxH1iyg9VD32j7f13w8zCqlaA4n/8d1CL2CkwEm4A8kD5s6B4ZykzMfZywxUw/lW6ZP3ajsNzyCb1upIrYBwCjV4R6+Ea1mao5UxQQOPi1dplz4/jdebMyUjazFkZXYmDylwbQ7aCxMCn94NAiG2dTRiBKhYvefy9rmf2TDTmesdj+l2ZSr9HdaWHoPfK5y/u3uf79Yx/KU0TEN3R5EiDEyQ6OMx44aoi7yT8joYWxg=='
Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and s